<a href="https://colab.research.google.com/github/devdoan/devdoan/blob/main/BANKER_GROUP_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def tinh_need(ma_tran_max, ma_tran_allocation):
    """Hàm tính ma trận Need (Nhu cầu còn lại)."""
    return ma_tran_max - ma_tran_allocation

def kiem_tra_trang_thai_an_toan(san_co, da_cap_phat, nhu_cau, so_tien_trinh, so_tai_nguyen):
    """
    Hàm kiểm tra xem hệ thống có đang ở trạng thái an toàn hay không.
    Trả về một tuple (là an toàn, chuỗi an toàn).
    """
    # 'work' là một bản sao của vector 'san_co', sẽ được cập nhật liên tục
    work = np.copy(san_co)
    # 'hoan_thanh' là một danh sách để theo dõi các tiến trình đã hoàn thành
    hoan_thanh = [False] * so_tien_trinh
    chuoi_an_toan = []

    # Vòng lặp sẽ tiếp tục cho đến khi tất cả các tiến trình được đưa vào chuỗi an toàn
    while len(chuoi_an_toan) < so_tien_trinh:
        tim_thay_tien_trinh = False
        for i in range(so_tien_trinh):
            # Tìm một tiến trình (i) chưa hoàn thành và có nhu cầu (need) nhỏ hơn hoặc bằng tài nguyên đang có (work)
            if not hoan_thanh[i] and np.all(nhu_cau[i] <= work):
                # Nếu tìm thấy, giả định tiến trình đó chạy xong và giải phóng tài nguyên đã được cấp phát
                work += da_cap_phat[i]
                hoan_thanh[i] = True
                chuoi_an_toan.append(f"P{i+1}")
                tim_thay_tien_trinh = True
                break

        # Nếu duyệt qua tất cả các tiến trình mà không tìm thấy tiến trình nào thỏa mãn, hệ thống không an toàn
        if not tim_thay_tien_trinh:
            return False, []

    # Nếu tất cả các tiến trình đều có thể hoàn thành, trả về trạng thái an toàn và chuỗi an toàn
    return True, chuoi_an_toan

def xu_ly_yeu_cau(id_tien_trinh, yeu_cau, san_co, da_cap_phat, nhu_cau, so_tien_trinh, so_tai_nguyen):
    """
    Hàm xử lý một yêu cầu tài nguyên từ một tiến trình.
    In ra quá trình quyết định và kết quả.
    """
    chi_so_p = id_tien_trinh - 1 # Chuyển ID (bắt đầu từ 1) thành chỉ số mảng (bắt đầu từ 0)
    print(f"--- Xử lý yêu cầu từ P{id_tien_trinh} cho {yeu_cau} ---")

    # Kiểm tra xem yêu cầu có vượt quá nhu cầu tối đa ban đầu không
    if not np.all(yeu_cau <= nhu_cau[chi_so_p]):
        print(f"LỖI: Yêu cầu {yeu_cau} vượt quá nhu cầu còn lại {nhu_cau[chi_so_p]}.")
        print(">> QUYẾT ĐỊNH: Từ chối yêu cầu.")
        return

    # Kiểm tra xem yêu cầu có vượt quá tài nguyên sẵn có của hệ thống không
    if not np.all(yeu_cau <= san_co):
        print(f"LỖI: Yêu cầu {yeu_cau} vượt quá tài nguyên sẵn có {san_co}.")
        print(">> QUYẾT ĐỊNH: Từ chối yêu cầu (tiến trình phải chờ).")
        return

    # Nếu yêu cầu hợp lệ, giả định cấp phát tài nguyên để kiểm tra
    print("Yêu cầu hợp lệ. Bắt đầu giả định cấp phát và kiểm tra an toàn...")

    san_co_gia_dinh = san_co - yeu_cau
    da_cap_phat_gia_dinh = np.copy(da_cap_phat)
    da_cap_phat_gia_dinh[chi_so_p] += yeu_cau
    nhu_cau_gia_dinh = np.copy(nhu_cau)
    nhu_cau_gia_dinh[chi_so_p] -= yeu_cau

    # Kiểm tra xem trạng thái mới có an toàn không
    trang_thai_an_toan, chuoi_an_toan_moi = kiem_tra_trang_thai_an_toan(san_co_gia_dinh, da_cap_phat_gia_dinh, nhu_cau_gia_dinh, so_tien_trinh, so_tai_nguyen)

    if trang_thai_an_toan:
        print(f"Trạng thái mới an toàn. Chuỗi an toàn tìm thấy: {chuoi_an_toan_moi}")
        print(">> QUYẾT ĐỊNH: Chấp nhận yêu cầu.")
    else:
        print("Trạng thái mới không an toàn. Việc cấp phát sẽ dẫn đến deadlock.")
        print(">> QUYẾT ĐỊNH: Từ chối yêu cầu (tiến trình phải chờ).")

    # --- Dữ liệu đầu vào của Nhóm 9 ---
so_tien_trinh = 3
so_tai_nguyen = 3

    # Ma trận Max (Nhu cầu tối đa của mỗi tiến trình)
ma_tran_max = np.array([
        [5, 3, 2],  # P1
        [4, 3, 2],  # P2
        [4, 2, 2]   # P3
    ])

    # Ma trận Allocation (Số tài nguyên đã cấp phát cho mỗi tiến trình)
ma_tran_allocation = np.array([
        [2, 1, 1],  # P1
        [2, 1, 1],  # P2
        [1, 2, 1]   # P3
    ])

    # Vector Available (Số tài nguyên đang sẵn có trong hệ thống)
vector_san_co = np.array([6, 5, 3])

    # Yêu cầu từ tiến trình P1
yeu_cau_p1 = np.array([1, 1, 0])

print("--- TRẠNG THÁI BAN ĐẦU CỦA HỆ THỐNG ---")
print("Ma trận Max:\n", ma_tran_max)
print("\nMa trận Allocation:\n", ma_tran_allocation)
print("\nVector Available:\n", vector_san_co)

    # --- Tính toán ma trận Need ---
ma_tran_need = tinh_need(ma_tran_max, ma_tran_allocation)
print("\n--- Ma trận Need (Max - Allocation) ---")
print("Ma trận Need:\n", ma_tran_need)
print("\n" + "="*50 + "\n")

    # --- Kiểm tra trạng thái an toàn ban đầu ---
print("--- Kiểm tra trạng thái an toàn ban đầu ---")
trang_thai_an_toan_ban_dau, chuoi_ban_dau = kiem_tra_trang_thai_an_toan(vector_san_co, ma_tran_allocation, ma_tran_need, so_tien_trinh, so_tai_nguyen)
if trang_thai_an_toan_ban_dau:
    print(f"Hệ thống đang ở trạng thái AN TOÀN.")
    print(f"Chuỗi an toàn tìm thấy: {chuoi_ban_dau}")
else:
    print("Hệ thống đang ở trạng thái KHÔNG AN TOÀN.")
print("\n" + "="*50 + "\n")

    # --- Xử lý yêu cầu ---
xu_ly_yeu_cau(1, yeu_cau_p1, vector_san_co, ma_tran_allocation, ma_tran_need, so_tien_trinh, so_tai_nguyen)


--- TRẠNG THÁI BAN ĐẦU CỦA HỆ THỐNG ---
Ma trận Max:
 [[5 3 2]
 [4 3 2]
 [4 2 2]]

Ma trận Allocation:
 [[2 1 1]
 [2 1 1]
 [1 2 1]]

Vector Available:
 [6 5 3]

--- Ma trận Need (Max - Allocation) ---
Ma trận Need:
 [[3 2 1]
 [2 2 1]
 [3 0 1]]


--- Kiểm tra trạng thái an toàn ban đầu ---
Hệ thống đang ở trạng thái AN TOÀN.
Chuỗi an toàn tìm thấy: ['P1', 'P2', 'P3']


--- Xử lý yêu cầu từ P1 cho [1 1 0] ---
Yêu cầu hợp lệ. Bắt đầu giả định cấp phát và kiểm tra an toàn...
Trạng thái mới an toàn. Chuỗi an toàn tìm thấy: ['P1', 'P2', 'P3']
>> QUYẾT ĐỊNH: Chấp nhận yêu cầu.
